In [1]:
from torchvision import transforms, utils, datasets
from torchvision.transforms import ToTensor
import torch

train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
    )


In [2]:
use_cuda = True

torch.manual_seed(23)
device = torch.device("cuda" if use_cuda else "cpu")
    # use_cuda가 true라면 kwargs를 다음과 같이 지정하기. 
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [26]:
import torch
from torch import nn
from torch.nn import functional as F

class MNIST(nn.Module) : #(1, 28, 28) -> (16, 12, 12) -> (32, 5, 5)
    def __init__(self) : 
        super(MNIST, self).__init__() 

        self.conv1 = nn.Conv2d(1,16, (5,5))
        self.conv1_normalize = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d((2,2))

        self.conv2 = nn.Conv2d(16,32, (3,3))
        self.conv2_normalize = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d((2,2))

        self.drop = nn.Dropout()
        self.fc1 = nn.Linear(800, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self,x) : 
        x = self.conv1(x) 
        x = self.conv1_normalize(x) 
        x = F.relu(x)
        x = self.pool1(x) 

        x = self.conv2(x)
        x = self.conv2_normalize(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.drop(x)
        x = x.view(-1, 800)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output 

In [27]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

CNN = MNIST().to(device)

optimizer = optim.Adam(CNN.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size = 1, gamma=0.01)

In [28]:
from torch.utils.data import  DataLoader
import torch.nn.functional as F
import torch

def train(batch_size, total_epoch, model, device, train_dataset, optimizer, epoch) : 
    model.train()
    data_loader = DataLoader(train_dataset, batch_size)

    for i, (data, label) in enumerate(data_loader) : 
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, label)
        loss.backward() 
        optimizer.step() 

        avg_loss = loss / len(data_loader)
        if i % 100 == 0 : 
            print('Train epoch : {} [{}/{} ({:.0f}%)]\t Loss: {:.4f}'. format(total_epoch, i, len(data_loader), 100. * i / len(data_loader), avg_loss))
    return model 


def test(batch_size, epoch, model, device, test_dataset, optimizer) : 
    model.eval() 
    test_loss = 0 
    correct = 0 

    data_loader = DataLoader(test_dataset, batch_size)

    with torch.no_grad() : 
        for (data, label) in data_loader : 
            data, label = data.to(device), label.to(device)

            output = model(data)
            test_loss += F.nll_loss(output, label, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim = True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    
    test_loss /= len(test_dataset)

    print('\n Test set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%) \n'.format(test_loss, correct, len(test_dataset), 100. *correct / len(test_dataset)))

    return correct/len(test_dataset)

In [29]:
from MNIST_train_test import train,

for epoch in range(1, 5):
        # train_test.py 의 train 함수를 통해 손 쉽게 모델 학습 진행 
        # train 수정 필요. 
    CNN = MNIST_train(50, 50, CNN, device, train_data, optimizer, epoch)
        
    scheduler.step()

        # train_test.py 의 test 함수를 통해 손 쉽게 모델 학습 진행 
    # 추후 pseudo labeling dataset에 대해서 바꿔줄 필요가 있음. 
accuracy = MNIST_test(50, 50, CNN, device, test_data, optimizer)
    

Train epoch : 50 [0/1200 (0%)]	 Loss: 0.0020
Train epoch : 50 [100/1200 (8%)]	 Loss: 0.0003
Train epoch : 50 [200/1200 (17%)]	 Loss: 0.0002
Train epoch : 50 [300/1200 (25%)]	 Loss: 0.0000
Train epoch : 50 [400/1200 (33%)]	 Loss: 0.0001
Train epoch : 50 [500/1200 (42%)]	 Loss: 0.0000
Train epoch : 50 [600/1200 (50%)]	 Loss: 0.0001
Train epoch : 50 [700/1200 (58%)]	 Loss: 0.0000
Train epoch : 50 [800/1200 (67%)]	 Loss: 0.0001
Train epoch : 50 [900/1200 (75%)]	 Loss: 0.0001
Train epoch : 50 [1000/1200 (83%)]	 Loss: 0.0001
Train epoch : 50 [1100/1200 (92%)]	 Loss: 0.0000
Train epoch : 50 [0/1200 (0%)]	 Loss: 0.0001
Train epoch : 50 [100/1200 (8%)]	 Loss: 0.0001
Train epoch : 50 [200/1200 (17%)]	 Loss: 0.0001
Train epoch : 50 [300/1200 (25%)]	 Loss: 0.0000
Train epoch : 50 [400/1200 (33%)]	 Loss: 0.0001
Train epoch : 50 [500/1200 (42%)]	 Loss: 0.0000
Train epoch : 50 [600/1200 (50%)]	 Loss: 0.0002
Train epoch : 50 [700/1200 (58%)]	 Loss: 0.0000
Train epoch : 50 [800/1200 (67%)]	 Loss: 0.000

In [30]:
accuracy

0.9852